In [1]:
# Using pysal-workshop env
import json, os, folium
import pandas as pd
import geopandas as gpd

In [2]:
sites_gdf = gpd.read_file("../data/covid_sites.geojson")
sites_gdf.shape

(158, 13)

In [3]:
sites_gdf.head()

,site_name,address,suburb,postcode,period,note,date_added,advice,tier,full_address,lat,lng,geometry
0,Dukes Gym Abbotsford,571-573 Victoria Street,Abbotsford,3067,24/05/2021 6:15pm - 8:15pm,Case attended venue,26/05/2021,Tier 1 - Get tested immediately and quarantin...,Tier 1,"571-573 Victoria Street , Abbotsford , Victo...",-37.81089,145.00567,POINT (145.00567 -37.81089)
1,Chelsea Heights Hotel,Springvale Road & Wells Road,Aspendale Gardens,3196,23/05/2021 12:10pm - 1:30pm,Case dined at venue,27/05/2021,Tier 1 - Get tested immediately and quarantin...,Tier 1,"Springvale Road & Wells Road , Aspendale Gar...",-38.02461,145.13410,POINT (145.13410 -38.02461)
2,Axedale Tavern,105 High Street,Axedale,3551,23/05/2021 11:45am - 1:30pm,Case attended function,26/05/2021,Tier 1 - Get tested immediately and quarantin...,Tier 1,"105 High Street , Axedale , Victoria, 3551 ...",-36.76132,144.27440,POINT (144.27440 -36.76132)
3,Endless Lifestyle Studio\r\n\r,Shop 3/188 Whitehorse Road\r\n\r,Balwyn,3103,23/05/2021 9:45am - 12:40pm,Case attended venue,26/05/2021,Tier 1 - Get tested immediately and quarantin...,Tier 1,"Shop 3/188 Whitehorse Road\r\n\r , Balwyn , ...",-37.81264,145.07602,POINT (145.07602 -37.81264)
4,Mcdonalds Bayswater North,296 Canterbury Road,Bayswater North,3153,21/05/2021 11:00pm - 12:00am,Case dined in restaurant,26/05/2021,Tier 1 - Get tested immediately and quarantin...,Tier 1,"296 Canterbury Road , Bayswater North , Vict...",-37.83149,145.27078,POINT (145.27078 -37.83149)


### Visualise on map

In [4]:
# OpenStreetMap
map = folium.Map(location = [-37.8251, 144.9509], tiles='OpenStreetMap' , zoom_start = 11)

In [5]:
# insert multiple markers, iterate through list
# add a different color marker associated with type of volcano

geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in sites_gdf.geometry ]
i = 0
for coordinates in geo_df_list:
    if sites_gdf.tier[i] == "Tier 1":
        type_color = "red"
    elif sites_gdf.tier[i] == "Tier 2":
        type_color = "orange"
    else:
        type_color = "blue"
    
    html = f'''
        <a href="https://www.coronavirus.vic.gov.au/exposure-sites"><b style="color:{type_color}">{sites_gdf.tier[i]}</b></a><br>
        Site name:<strong> {sites_gdf.site_name[i]}</strong><br>
        Address: {sites_gdf.address[i]}, {sites_gdf.suburb[i]}<br>
        Postcode: {sites_gdf.postcode[i]}<br>
        Period: {sites_gdf.period[i]}<br>
        
        <sub style="color:{type_color};"><b>Health advice:</b> {sites_gdf.advice[i]}</sub>
        '''
    iframe = folium.IFrame(html, width=300, height=150)
    popup = folium.Popup(iframe, max_width=300)
    #now place the markers with the popup labels and data
    map.add_child(folium.Marker(location = coordinates,
                            popup = popup, 
                            icon = folium.Icon(color = "%s" % type_color)))
    i = i + 1

In [6]:
map

In [7]:
# Save map to html and the deploy on Github page
map.save("../covid19_map.html")